### Exercice 3 :
##### Quel est le travail d'un système de correction automatique ?
Un système de correction automatique change un mot mal orthographié en son orthographe correcte.
##### Étapes pour la mise en place d'un système de correction automatique :
1. Identifier le mot mal orthographié.
2. Trouver des chaînes de caractères qui sont à n distances d'édition du mot mal orthographié.
3. Filtrer les candidats suggérés pour ne retenir que ceux trouvés dans le vocabulaire.
4. Classer les candidats filtrés en fonction des probabilités des mots.
5. Choisir le candidat le plus probable.
##### Identifier un mot mal orthographié : 
Un mot est mal orthographié s'il n'est pas trouvé dans le vocabulaire du corpus de texte avec lequel le système de correction automatique travaille .
##### Trouver des chaînes qui sont à n distances d'édition : 
L'édition est une opération effectuée sur une chaîne de caractères pour la changer en une autre. Une distance d'édition est un comptage du nombre d'opérations effectuées sur un mot pour l'éditer.
##### Types d'opérations d'édition :
* INSERT 
* DELETE 
* SWAP
* REPLACE 
##### Calcul des probabilités des mots : 
Les probabilités d'un mot sont calculées selon la formule suivante : P(w)= C(w)/V 
où:
* P(w) est la probabilité d'un mot w
* C(w) est le nombre de fois qu'un mot w apparaît dans le corpus
* V est le nombre total de mots dans le corpus
##### Minimun Edit Distance ( MED): 

La distance d'édition minimale est le nombre minimal de modifications nécessaires pour transformer une chaîne de caractères en une autre.

 

In [1]:
import re
import string
from collections import Counter
import numpy as np

### Question 1 : 
process_data lit un fichier texte, extrait tous les mots, les convertit en minuscules pour assurer la cohérence, et retourne une liste de ces mots.

In [2]:
def process_data(corpus_file):
  with open(corpus_file, "r") as file:
    lines = file.readlines()
    words = []
    for line in lines:
      words += re.findall(r'\w+', line.lower())

  return words

In [3]:
words = process_data("big.txt")
print(f"There are {len(words)} total words in the corpus")

There are 1115585 total words in the corpus


#### Question 2 :
la fonction get_vocabulary retourne le vocabulaire construit à partir d’un corpus passé en argument de la fonction.

In [10]:
def get_vocabulary(corpus_file):
    words = process_data(corpus_file)
    vocabs = set(words)
    return vocabs

In [11]:
vocabulary = get_vocabulary("big.txt")
print(f"There are {len(vocabulary)} unique words in the vocabulary")

There are 32198 unique words in the vocabulary


#### Question 3 :
build_language_model construit et retourne un modèle de langue qui représente la probabilité d'apparition de chaque mot dans un corpus donné, basé sur la fréquence de ces mots dans le corpus.

In [12]:
def build_language_model(corpus_file):
    words = process_data(corpus_file)
    word_counts = Counter(words)
    total_word_count = float(sum(word_counts.values()))
    word_probas = {word: word_counts[word] / total_word_count for word in word_counts.keys()}
    return word_probas

In [14]:
language_model = build_language_model("big.txt")
print(language_model["dog"])

5.736900370657547e-05


#### Question 4 :
Implémentation des fcts : edits1 , edits2 et knownWord

In [15]:
# edit1

def edits1(word):
    """
    Générer toutes les chaînes à une modification (insertion, suppression, substitution) de distance du mot `word`.
    """
    def split(word):
        return [(word[:i], word[i:]) for i in range(len(word) + 1)]

    def delete(word):
        return [L + R[1:] for L, R in split(word) if R]

    def swap(word):
        return [L + R[1] + R[0] + R[2:] for L, R in split(word) if len(R) > 1]

    def replace(word):
        letters = string.ascii_lowercase
        return [L + c + R[1:] for L, R in split(word) if R for c in letters]

    def insert(word):
        letters = string.ascii_lowercase
        return [L + c + R for L, R in split(word) for c in letters]

    return set(delete(word) + swap(word) + replace(word) + insert(word))


In [17]:
# Exemple d'utilisation de la fonction edits1
edits1_exemple = "cat"
modif = edits1(edits1_exemple)

print("Quelques modifications à une distance de 'cat':")
print(list(modif)[:10])  # Afficher les 10 premières modifications pour l'exemple

Quelques modifications à une distance de 'cat':
['qat', 'oat', 'cav', 'cats', 'vat', 'cabt', 'cal', 'catt', 'capt', 'ncat']


In [18]:
#edits2
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))


In [19]:
# Exemple d'utilisation de la fonction edits2
edits2_exemple = "cat"
modif2 = edits2(edits2_exemple)

print("Quelques modifications à deux distances de 'cat':")
print(list(modif2)[:10])  # Afficher les 10 premières modifications pour l'exemple


Quelques modifications à deux distances de 'cat':
['caeh', 'citd', 'cozat', 'cavb', 'cakts', 'ckta', 'cvtw', 'cuo', 'wamt', 'cfatq']


In [35]:
# knownWord : filtrer une liste de mots pour ne garder que ceux présents dans le vocabulaire.
def knownWord( words):
    return set(w for w in words if w in vocabulary)


##### Question 5 : 
La fonction candidates(word) pour générer des candidats de correction pour un mot donné.
* param word: Mot à corriger
* return: Ensemble de candidats pour la correction

In [40]:
def candidates(word):
    return (knownWord([word]) or knownWord(edits1(word)) or knownWord(edits2(word)) or [word])

##### Question 6 :
La fct correction(word,k ) pour trouver la ou les meilleures corrections pour un mot donné.
* param word: Mot à corriger
* param k: Nombre maximal de suggestions à retourner
* return: Liste des meilleures corrections

In [43]:
def correction(word, k=1):
    "Return the most probable spelling correction for word."
    candidates = knownWord([word]) or knownWord(edits1(word)) or knownWord(edits2(word)) or [word]
    best_guesses = sorted(candidates, key=language_model.get, reverse=True)[:k]
    return best_guesses

In [44]:
correction('speling')

['spelling']

In [46]:
correction('korrectud')

['corrected']